In [1]:
# Load embeddings
from pickle import load
import numpy as np
from PIL import Image
import os

image_folder = "images/"

# Break into categories
clothing_images = [Image.open(os.path.join(image_folder, file)) for file in os.listdir(image_folder) if file.endswith("c.png")]
clothing_labels = [file.split("-")[0].split("_") for file in os.listdir(image_folder) if file.endswith("c.png")]
clothing_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in clothing_labels]

full_body_images = [Image.open(os.path.join(image_folder, file)) for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_labels = [file.split("-")[0].split("_") for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_filenames = [file for file in os.listdir(image_folder) if file.endswith("fb.png")]
full_body_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in full_body_labels]

In [2]:
# # Preprocess embedings
# clothing_genders = np.zeros((clothing_embed.shape[0], 2))
# clothing_genders[np.array([{"mens": 0, "womens": 1}[lab[1]] for lab in clothing_labels])] = 1

# clothing_type = np.zeros((clothing_embed.shape[0], 6))
# clothing_type[np.array([
#     {"shirt": 0, "top": 1, "sweater": 2, "pant": 3, "skirt": 4, "short": 5}[lab[2]]
#     for lab in clothing_labels
# ])] = 1

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

tf.config.list_physical_devices()

2024-04-30 20:42:06.143533: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 20:42:06.187976: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 20:42:06.798519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [4]:
# Train model
import tensorflow as tf

class triplet_model(tf.keras.Model):
    def __init__(self, model, output_size, margin=0, trainable=True,):
        super().__init__(trainable=trainable)
        self.clothing_model = tf.keras.models.Sequential([
            model,
            tf.keras.layers.Dense(output_size * 4, activation="elu"),
            tf.keras.layers.Dense(output_size * 4, activation="elu"),
            tf.keras.layers.Dense(output_size, activation="sigmoid"),
        ])
        self.outfit_model = tf.keras.models.Sequential([
            model,
            tf.keras.layers.Dense(output_size * 4, activation="elu"),
            tf.keras.layers.Dense(output_size * 4, activation="elu"),
            tf.keras.layers.Dense(output_size, activation="sigmoid"),
        ])
        self.margin = margin

    def compile(self, **kwargs):
        self.clothing_model.compile(**kwargs)
        self.outfit_model.compile(**kwargs)
        super().compile(**kwargs)

    # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#supporting_sample_weight_class_weight
    # https://en.wikipedia.org/wiki/Triplet_loss
    def train_step(self, data):
        x, x_positive, x_negative = data

        with tf.GradientTape() as tape:
            y_anchor = self.clothing_model(x)
            y_positive = self.outfit_model(x_positive)
            y_negative = self.outfit_model(x_negative)
            
            dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
            dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_negative), axis=-1))
            loss = tf.math.maximum(dist_positive - dist_negative + self.margin, 0)
            loss = tf.math.reduce_sum(loss)

        # Compute gradients
        trainable_vars = self.clothing_model.trainable_variables + self.outfit_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, x_positive, x_negative = data
        
        y_anchor = self.clothing_model(x)
        y_positive = self.outfit_model(x_positive)
        y_negative = self.outfit_model(x_negative)
        
        dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
        dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_negative), axis=-1))
        loss = tf.math.maximum(dist_positive - dist_negative + self.margin, 0)
        loss = tf.math.reduce_sum(loss)

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def triplet_sample_generator(validation=False):
    if validation:
        inds = np.arange(len(clothing_images))[np.arange(len(clothing_images)) % 10 == 0]
    else:
        inds = np.arange(len(clothing_images))[np.arange(len(clothing_images)) % 10 != 0]

    for i in inds:
        neg_inds = inds[inds != i]
        np.random.shuffle(neg_inds)

        anchor = tf.keras.applications.mobilenet_v3.preprocess_input(np.array(clothing_images[i].resize((224, 224))))
        positive = tf.keras.applications.mobilenet_v3.preprocess_input(np.array(full_body_images[i].resize((224, 224))))
        negative = tf.keras.applications.mobilenet_v3.preprocess_input(np.array(full_body_images[neg_inds[0]].resize((224, 224))))
        
        yield anchor, positive, negative

img_spec = tf.TensorSpec((224, 224, 3), float)
ds_train = tf.data.Dataset.from_generator(triplet_sample_generator, output_signature=(img_spec, img_spec, img_spec))
ds_valid = tf.data.Dataset.from_generator(lambda: triplet_sample_generator(validation=True), output_signature=(img_spec, img_spec, img_spec))

model = tf.keras.applications.MobileNetV3Large(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg"
)
inp = tf.keras.Input((224, 224, 3))
model = tf.keras.Model(inputs=inp, outputs=model(inp))
model.summary()

model = triplet_model(
    model,
    512,
    margin = 1
)

model.compile(
    optimizer="adam",
    run_eagerly=False,
    metrics=["binary_crossentropy", "binary_accuracy"]
)

model.fit(
    ds_train.batch(32),
    epochs=100,
    validation_data=ds_valid.batch(32),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=25, restore_best_weights=True)],
)   

2024-04-30 20:42:07.757342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22253 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:18:00.0, compute capability: 8.9
2024-04-30 20:42:07.757840: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22446 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobilenetV3large (Functional)   │ (None, 960)            │     2,996,352 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,996,352 (11.43 MB)

 Trainable params: 2,971,952 (11.34 MB)

...

2024-04-30 20:51:54.192528: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-30 20:51:54.192614: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-04-30 20:51:54.192645: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11500698178648149573


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 764ms/step - binary_accuracy: 0.0763 - binary_crossentropy: 14.8881 - loss: 31.9388

2024-04-30 20:52:08.182038: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-30 20:52:08.183640: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 869ms/step - binary_accuracy: 0.0767 - binary_crossentropy: 14.8818 - loss: 31.8974 - val_binary_accuracy: 0.0794 - val_binary_crossentropy: 14.8389 - val_loss: 31.4854


2024-04-30 20:52:09.929863: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-30 20:52:09.929935: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_6]]
2024-04-30 20:52:09.929964: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11500698178648149573


In [5]:
# Test model
converted_clothing_embeds = model.clothing_model(tf.keras.applications.mobilenet_v3.preprocess_input(
    np.array([img.resize((224, 224)) for img in clothing_images[:8]])
))
converted_clothing_embeds = tf.expand_dims(converted_clothing_embeds, axis=1)
print(converted_clothing_embeds)

converted_outfit_embeds = model.outfit_model(tf.keras.applications.mobilenet_v3.preprocess_input(
    np.array([img.resize((224, 224)) for img in full_body_images[:8]])
))
converted_outfit_embeds = tf.expand_dims(converted_outfit_embeds, axis=0)
print(converted_outfit_embeds)

distances = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(converted_outfit_embeds - converted_clothing_embeds), axis=-1))
best_samples = tf.math.top_k(distances, k=5).indices

best_samples

tf.Tensor(
[[[0. 0. 0. ... 1. 1. 0.]]

 [[0. 0. 0. ... 1. 1. 0.]]

 [[0. 0. 0. ... 1. 1. 0.]]

 ...

 [[0. 0. 0. ... 1. 1. 0.]]

 [[0. 0. 0. ... 1. 1. 0.]]

 [[0. 0. 0. ... 1. 1. 0.]]], shape=(8, 1, 512), dtype=float32)
tf.Tensor(
[[[1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  ...
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]]], shape=(1, 8, 512), dtype=float32)


<tf.Tensor: shape=(8, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]], dtype=int32)>

In [6]:
full_body_filenames[7]

'jcrew_womens_tops-BV829-YD4884-fb.png'